# Symbolic Derivatives of Normal Python Functions

Consider this innocuous-looking integer-to-integer function.

In [15]:
def f(x):
    return x * 2 + x * x

Recall some of the basic rules for computing derivatives from calculus.  (Or, if you've never seen them before, no need to worry, because we will just apply these rules mechanically!)  We explain how to transform each kind of expression into an expression for the derivative with respect to `x`.

`D[x] = 1`

`D[k] = 0`, for `k` a constant

`D[e1 + e2] = D[e1] + D[e2]`

`D[e1 * e2] = D[e1]*e2 + e1*D[e2]`

How can we apply these rules automatically to Python functions like the above, to compute their derivatives in *symbolic* form that we can print for easy inspection?  It's not as easy as it sounds!

However, Python's *magic methods* will provide a solution!

Here's us cutting right to the solution, which we develop incrementally in lecture.  The basic idea is to develop a library of classes, standing for different kinds of arithmetic expressions over one variable `x`.  Afterward, we show how to use these classes for easy calculation of symbolic derivatives of normal Python functions.

In [16]:
# First, we define a base class for expressions.
# That is, any expression will provide these methods with these bodies.
class Expr:
    # Magic method #1: an __add__ method explains how to handle use of the '+' operator
    # when the *first* operand belongs to this class, thereby being passed in as self.
    # The other operand is passed as the second argument.
    def __add__(self, e):
        # If we detect the second argument is a number, replace it with an instance of
        # our class for constants, so that both operands can be Exprs.
        if isinstance(e, int):
            e = Const(e)

        # Now return an instance of our class for additions.
        return Plus(self, e)
    
    # Everything is symmetrical for multiplication.
    def __mul__(self, e):
        if isinstance(e, int):
            e = Const(e)

        return Times(self, e)

# Our class for constants, which *inherits* from Expr
class Const(Expr):
    def __init__(self, num):
        self.num = num
    
    # Another magic method: we dictate what it means to call an expression like a function!
    # What do we say it means?  It's just evaluating the expression on a particular x value.
    def __call__(self, x):
        return self.num
    
    # We also provide easy symbolic construction of derivatives,
    # following the rules quoted above.
    def derivative(self):
        return Const(0)
    
    # It is handy to return a version of an expression that may have been simplified algebraically.
    # However, no simplifications apply to constants in isolation.
    def simplify(self):
        return self
    
    # This magic method is for generating the string that appears when we print an object.
    def __repr__(self):
        return str(self.num)

# The variable x
class Var(Expr):
    def __call__(self, x):
        return x
    
    def derivative(self):
        return Const(1)
    
    def simplify(self):
        return self
    
    def __repr__(self):
        return "x"

# Addition
class Plus(Expr):
    def __init__(self, e1, e2):
        self.e1 = e1
        self.e2 = e2
    
    def __call__(self, x):
        return self.e1(x) + self.e2(x)
    
    def derivative(self):
        return self.e1.derivative() + self.e2.derivative()
    
    def simplify(self):
        e1 = self.e1.simplify()
        e2 = self.e2.simplify()
        
        if isinstance(e1, Const) and e1.num == 0:
            # Applies the law: 0 + y = y
            return e2
        elif isinstance(e2, Const) and e2.num == 0:
            # y + 0 = y
            return e1
        elif isinstance(e1, Const) and isinstance(e2, Const):
            # This case just pushes addition inside the Const class.
            return Const(e1.num + e2.num)
        else:
            return e1 + e2
    
    def __repr__(self):
        return "(" + self.e1.__repr__() + " + " + self.e2.__repr__() + ")"

class Times(Expr):
    def __init__(self, e1, e2):
        self.e1 = e1
        self.e2 = e2
    
    def __call__(self, x):
        return self.e1(x) * self.e2(x)
    
    def derivative(self):
        return self.e1.derivative() * self.e2 + self.e1 * self.e2.derivative()
    
    def simplify(self):
        e1 = self.e1.simplify()
        e2 = self.e2.simplify()
        
        if isinstance(e1, Const) and e1.num == 1:
            # 1 * y = y
            return e2
        elif isinstance(e2, Const) and e2.num == 1:
            # y * 1 = 1
            return e1
        elif isinstance(e1, Const) and e1.num == 0:
            # 0 * y = 0
            return Const(0)
        elif isinstance(e2, Const) and e2.num == 0:
            # y * 0 = 0
            return Const(0)
        elif isinstance(e1, Const) and isinstance(e2, Const):
            # Push multiplication inside the Const class.
            return Const(e1.num * e2.num)
        else:
            return e1 * e2
    
    def __repr__(self):
        return "(" + self.e1.__repr__() + " * " + self.e2.__repr__() + ")"

We can pretty easily build syntax trees and evaluate them on `x` values.

In [17]:
e = Plus(Times(Var(), Const(2)), Times(Var(), Var()))
e(3)

15

However, using the overloaded addition and multiplication operators we defined within `Expr`, we can write an even more satisfying syntax.

In [18]:
e = Var() * 2 + Var() * Var()
e(3)

15

Computing derivates is easy, and the output is even nice-looking thanks to our `__repr__` methods.

In [19]:
e.derivative()

(((1 * 2) + (x * 0)) + ((1 * x) + (x * 1)))

However, we might want to simplify the derivative before printing it.

In [20]:
e.derivative().simplify()

(2 + (x + x))

Back to our original puzzle: how do we compute symbolic derivatives of normal Python functions like `f` defined above?  Answer: apply that function *once* to a `Var` instance, letting our operator overloading do the rest of the work!

In [21]:
def expr_of(f):
    return f(Var())

In [22]:
expr_of(f)

((x * 2) + (x * x))

Nice: we got out the perfect syntax tree.  From here, it's easy to compute and display the derivative.

In [23]:
expr_of(f).derivative().simplify()

(2 + (x + x))

Did you catch one weakness of the approach?  What if we try to differentiate the following functions?

In [24]:
def g(x):
    return 2 * x + x * x

Looks the same in all the ways we care about, right?  We just took advantage of commutativity of multiplication.

In [14]:
expr_of(g).derivative().simplify()

TypeError: unsupported operand type(s) for *: 'int' and 'Var'

Agh, a scary error message!  The reason is that `*` decides which code to call based on the type of its *first* operand, and here we have a number as the first operand.  The built-in `int` class knows nothing about our syntax trees.

It turns out here we need the *reverse multiply* magic method, and, for good measure, we'll show how to add it to the `Expr` class after the fact.  (This kind of "method patching" leads to hard-to-understand code, so we don't recommend it for most situations.)

In [ ]:
def expr_radd(self, e):
    if isinstance(e, int):
        e = Const(e)

    return Plus(e, self)
    
Expr.__radd__ = expr_radd

def expr_rmul(self, e):
    if isinstance(e, int):
        e = Const(e)

    return Times(e, self)
    
Expr.__rmul__ = expr_rmul

In [ ]:
expr_of(g).derivative().simplify()

(2 + (x + x))

# Roll-Your-Own Generators

We looked at generators a bit in Lecture 6, for iterating over the nodes of a tree.  Generators bring two main advantages over lists:
 1. With a generator, we avoid needing to store all generated values simultaneously, which can bring significant memory savings.
 2. In fact, it might be *impossible* to store all values generated!  A generator may yield *infinitely* many values.

Since we usually want our programs to finish, how could it be useful to generate infinitely many values?  Check out this example.  We will take a roundabout path to finding the first few prime numbers.

In [25]:
def all_positive_integers():
    """Generator for all numbers greater than zero"""
    
    i = 1
    while True:
        # Note: loop is intentionally nonterminating!
        yield i
        i += 1

def firstn(stream, num):
    """Return the first num (which must be positive) values generated by stream."""
    
    for v in stream:
        yield v
        num -= 1
        if num <= 0:
            return

Yes, a naive reading of `all_positive_integers` shows it running forever.  However, it continually invokes `yield`, which *suspends execution of the generator, until the consumer is ready for the next value*.  A consumer that only asks for finitely many values can terminate, even when the generator has the *capacity* to produce infinitely many values.  Here are some examples of pulling out interesting subsequences of `all_positive_integers`.

In [ ]:
list(firstn(all_positive_integers(), 10))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
list(firstn((n for n in all_positive_integers() if n % 2 == 0), 10))

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [ ]:
list(firstn((n for n in all_positive_integers() if n % 7 == 0), 10))

[7, 14, 21, 28, 35, 42, 49, 56, 63, 70]

We can also implement primality testing to get the first few primes.

In [ ]:
def is_prime(n):
    return n > 1 and all(n % m != 0 for m in range(2, n))

list(firstn((n for n in all_positive_integers() if is_prime(n)), 20))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

## Replicating the Functionality Without Using Generators

It may seem like magic, how Python lets us code up an infinite sequence of values.  Let's reveal how straightforward it all is, by building the same features from basic classes.  (Our version won't look as nice, but it's equivalent in any deep sense.)

In [ ]:
# Everywhere we used a generator, we'll instead use a child of this class.
class Iterator:     
    def hasNext(self):
        """Intended behavior: returns whether there is a next element."""
        raise NotImplementedError
    
    def next(self):
        """Intended behavior: returns next element.
        Illegal to call when hasNext() returns False!"""
        raise NotImplementedError
    
    # Bonus: we use yet another magic method to build a native Python iterator
    # from one of our hand-crafted ones.
    def __iter__(self):
        while self.hasNext():
            yield self.next()

Now here's how to duplicate the generation of all positive integers.

In [ ]:
class AllPositiveIntegers(Iterator):
    def __init__(self):
        self.i = 1 # Attribute to hold next integer we should generate

    def hasNext(self):
        return True # Infinite streams always have next values!

    def next(self):
        result = self.i
        self.i += 1
        return result

In [ ]:
api = AllPositiveIntegers()
print(api.next())
print(api.next())
print(api.next())

1
2
3


Now let's redo the first-n functionality.

In [ ]:
class FirstN(Iterator):
    def __init__(self, child, num):
        self.child = child # A sub-iterator that we delegate to
        self.num = num     # How many more values we want to grab from it, max
    
    def hasNext(self):
        return self.num > 0 and self.child.hasNext()
        # We must want to return more values AND the child must actually have a value to offer.
    
    def next(self):
        self.num -= 1
        return self.child.next()

In [ ]:
list(FirstN(AllPositiveIntegers(), 2))

[1, 2]

Note that the `list` call above worked implicitly by calling the `__iter__` method of `Iterator`, which is called in any context where Python wants an iterator (in the built-in sense) but has been handed a value that isn't one yet.

We can also easily go in the other direction, in converting between native iterators and our explicit ones.  This class's constructor takes a native iterator as an argument.

In [ ]:
class FromNativeIterator(Iterator):
    def __init__(self, child):
        self.child = iter(child)    # We delegate to this native iterator,
                                    # calling iter to force it to be an actual iterator.
        self.nextValue = None       # Stores a value confirmed to be next from the child.
        self.unclaimedNext = False  # Records whether nextValue is in use or not.
    
    def hasNext(self):
        if self.unclaimedNext:
            return True
        
        # Note: we run a "loop" here designed so that it only runs for 0 or 1 iterations.
        for v in self.child:
            # Aha, there is a next value!  Store it away for next to grab.
            self.nextValue = v
            self.unclaimedNext = True
            return True
        
        # No values available from the child.
        return False

    def next(self):
        if self.hasNext():
            # Just pull out and return the value we saved, erasing it.
            result = self.nextValue
            self.unclaimedNext = False
            self.nextValue = None
            return result

In [ ]:
r = FromNativeIterator(range(0, 3))
print(r.hasNext())
print(r.next())
print(r.hasNext())
print(r.next())
print(r.hasNext())
print(r.next())
print(r.hasNext())

True
0
True
1
True
2
False


The following invocation creates a list by creating the `range` iterator, converting it to our manual style, then implicitly converting back to a native iterator via `Iterator.__iter__` (since built-in function `list` wants its argument to be iterable).

In [ ]:
list(FromNativeIterator(range(0, 3)))

[1, None]

## Comprehensions

Here's a list comprehension suitable for two-dimensional minesweeper from Lab 3.

In [ ]:
def coordinate_in_bounds(x, y, max_x, max_y):
    """Is point (x, y) in bounds in a two-dimensional space with
    the given maximum coordinates and minimum coordinates zero?"""
    return 0 <= x and x <= max_x and 0 <= y and y <= max_y

def neighbors(x, y, max_x, max_y):
    """Return list of coordinates of all neighbors of (x, y) in a 2-dimensionsal space
    with the given maximum coordinate values."""
    
    return [(nx, ny)
            for nx in range(x - 1, x + 2)
            for ny in range(y - 1, y + 2)
            if coordinate_in_bounds(nx, ny, max_x, max_y)
            if (nx, ny) != (x, y)]

In [ ]:
print(neighbors(0, 0, 2, 2))
print(neighbors(1, 1, 2, 2))
print(neighbors(2, 1, 2, 2))

[(0, 1), (1, 0), (1, 1)]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (2, 2)]
[(1, 0), (1, 1), (1, 2), (2, 0), (2, 2)]


How do comprehensions really work?  Python translates them into generators automatically!  Let's first build equivalent code in a very manual way, using our `Iterator` class.

First, we replicate the built-in `range` function.

In [ ]:
class Range(Iterator):
    def __init__(self, lower, upper):
        self.lower = lower
        self.upper = upper
    
    def hasNext(self):
        return self.lower < self.upper

    def next(self):
        result = self.lower
        self.lower += 1
        return result

In [ ]:
list(Range(3, 6))

[3, 4, 5]

Now a trickier case: a *composed* iterator takes in an iterator and *a function that creates iterators*.  The meaning is: loop through the first iterator, calling the function on each value as it is ready, performing a nested loop over the iterator that the function returns.

In [ ]:
class Composed(Iterator):
    def __init__(self, iter1, iter2):
        self.iter1 = iter1
        self.iter2 = iter2
        self.specific_iter2 = None # This attribute holds the most recent iterator
                                   # created by the function iter2.
    
    def hasNext(self):
        # It is a bit tricky to tell when there is a next item.
        # We might need to "fast-forward" through many values produced by iter1,
        # if each one corresponds to an empty iter2.
        # As a nice side effect, we stash away the next value to return, if there is one.
        while True:
            if self.specific_iter2 and self.specific_iter2.hasNext():
                return True
        
            if self.iter1.hasNext():
                #keep going to next loop (advancing iter1) until specic_iter2 hasNext
                self.specific_iter2 = self.iter2(self.iter1.next())
            else:
                return False
    
    def next(self):
        if self.hasNext():
            return self.specific_iter2.next()

In [ ]:
list(Composed(Range(0, 4), lambda n: Range(n, n+2)))

[0, 1, 1, 2, 2, 3, 3, 4]

A *singleton* iterator yields just one value, which we give up-front.

In [ ]:
class Singleton(Iterator):
    def __init__(self, value):
        self.value = value     # Save to return when asked.
        #this means we cannot run next on this item multiple times
        self.unclaimed = True  # Remember whether we were already asked.
    
    def hasNext(self):
        return self.unclaimed

    def next(self):
        #claim (mark this value as used before) this item
        self.unclaimed = False
        return self.value

In [ ]:
list(Composed(Range(0, 4), lambda n: Singleton(n*n)))

[0, 1, 4, 9]

A *guard* iterator enforces a condition, yielding one dummy value if the condition holds, otherwise yielding no values.

In [ ]:
class Guard(Iterator):
    def __init__(self, condition):
        self.unclaimed = condition
    
    def hasNext(self):
        return self.unclaimed
    
    def next(self):
        self.unclaimed = False
        return () # The dummy value is an empty tuple.

In [ ]:
list(Composed(Range(0, 4), lambda n: Guard(n % 2 == 0)))

[(), ()]

We are almost ready to replicate our comprehension example for enumerating neighbors.  We just need a way to convert an iterator to a list.  (Previously we used the `list` function, but it relies on conversion to built-in iterators, which seems like cheating!)

In [ ]:
def list_of(iter1):
    """Return list of all elements generated by iter1."""
    
    ls = []
    while iter1.hasNext():
        ls.append(iter1.next())
    return ls

In [ ]:
def neighbors(x, y, max_x, max_y):
    """Return list of coordinates of all neighbors of (x, y) in a 2-dimensionsal space
    with the given maximum coordinate values."""
    
    return Composed(Range(x - 1, x + 2),
                    lambda nx: Composed(Range(y - 1, y + 2),
                    lambda ny: Composed(Guard(coordinate_in_bounds(nx, ny, max_x, max_y)),
                    lambda _: Composed(Guard((nx, ny) != (x, y)),
                    lambda _: Singleton((nx, ny))))))

def neighbor_list(x, y, max_x, max_y):
    return list_of(neighbors(x, y, max_x, max_y))

In [ ]:
neighbor_list(0, 0, 2, 2)

[(0, 1), (1, 0), (1, 1)]

In [ ]:
neighbor_list(1, 1, 2, 2)

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (2, 2)]

In [ ]:
neighbor_list(2, 2, 2, 2)

[(1, 1), (1, 2), (2, 1)]

It's also easy to produce an answer as a set rather than a list.

In [ ]:
def set_of(iter1):
    """Return set of all elements generated by iter1."""
    
    s = set()
    while iter1.hasNext():
        s.add(iter1.next())
    return s

def neighbor_set(x, y, max_x, max_y):
    return set_of(neighbors(x, y, max_x, max_y))

In [ ]:
neighbor_set(1, 1, 2, 2)

{(0, 0), (0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (2, 2)}

Compare the code with how we write the same computation using Python's built-in generators.

In [ ]:
def neighbors_native(x, y, max_x, max_y):
    """Return list of coordinates of all neighbors of (x, y) in a 2-dimensionsal space
    with the given maximum coordinate values."""
    
    for nx in range(x - 1, x + 2):
        for ny in range(y - 1, y + 2):
            if coordinate_in_bounds(nx, ny, max_x, max_y):
                if (nx, ny) != (x, y):
                    yield (nx, ny)

In [ ]:
list(neighbors_native(1, 1, 2, 2))

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (2, 2)]

# Reflection

*Encapsulation*, e.g. as we studied it in Lecture 7, is one of the key foundations of modern software development.  We define classes and other units of code with *private state* that other parts of a program aren't allowed to see.

However, sometimes we can implement some pretty cool functionality by peeking inside the definitions of things!  The term of art for said peeking is *reflection*.

To work up to some examples, consider this definition of a *queue* data structure, where we can *enqueue* values, which are then *dequeued* in the same order as we added them.

In [26]:
class BasicQueue:
    def __init__(self):
        self.contents = []
    
    def enqueue(self, v):
        self.contents.append(v)
    
    def dequeue(self):
        if self.contents == []:
            raise ValueError
        else:
            result = self.contents[0]
            self.contents = self.contents[1:]
            # Note the previous line can be rather slow,
            # as it must recreate most of the contents list,
            # in time proportional to the list length.
            return result

In [27]:
q = BasicQueue()
q.enqueue(1)
q.enqueue(2)
print(q.dequeue())
q.enqueue(3)
print(q.dequeue())
print(q.dequeue())

1
2
3


Here is another implementation, which manages to avoid recreating a list on every dequeue.  We won't dwell on details of the code; what matters is it's a less-obvious implementation of the same functionality from `BasicQueue`.

In [28]:
class FancyQueue:
    def __init__(self):
        self.enqueueHere = []
        self.dequeueHere = []
    
    def enqueue(self, v):
        self.enqueueHere.append(v)
    
    def dequeue(self):
        if self.dequeueHere == []:
            if self.enqueueHere == []:
                raise ValueError
            else:
                self.dequeueHere = self.enqueueHere
                self.enqueueHere = []
                self.dequeueHere.reverse()
        
        return self.dequeueHere.pop()

In [29]:
q = FancyQueue()
q.enqueue(1)
q.enqueue(2)
print(q.dequeue())
q.enqueue(3)
print(q.dequeue())
print(q.dequeue())

1
2
3


Good: it returned the same answers as `BasicQueue`!  But that was just one test case.  Can we write some code to help us run many comparison tests easily?  Ideally, we would write *generic* code that builds testing infrastructure for a wide variety of classes, not just queue classes.  Reflection will let us do exactly that.

First, we introduce the `dir` function, for listing the methods of a class.

In [30]:
dir(BasicQueue)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'dequeue',
 'enqueue']

The long list reveals that all Python classes inherit a variety of magic methods, which will not interest us here.  We can easily filter them out, thanks to their naming convention.

In [31]:
[name for name in dir(BasicQueue) if not name.startswith('__')]

['dequeue', 'enqueue']

Good: now we only see the methods we really want to test.  How do we go from these names back to the real methods?  Like so, with built-in function `getattr`:

In [32]:
getattr(BasicQueue, 'dequeue')

<function __main__.BasicQueue.dequeue(self)>

That's a good start, but we also need a way to list the arguments of a method.  Here we break the cardinal rule of 6.009 labs and import one function from a built-in library!

In [33]:
from inspect import signature
signature(getattr(BasicQueue, 'dequeue')).parameters

mappingproxy({'self': <Parameter "self">})

In [34]:
signature(getattr(BasicQueue, 'enqueue')).parameters

mappingproxy({'self': <Parameter "self">, 'v': <Parameter "v">})

That's a bit of a weird output.  Think of this function as returning dictionaries (from parameter names to info on them), but wrapped in a strange way.  Below we will immediately convert such values into normal dictionaries with the `dict` function.

The last ingredient we need is a way of calling methods with argument lists that we construct as dictionaries.

In [299]:
q = BasicQueue()
getattr(q, 'enqueue')(**{'v': 3})
q.dequeue()

3

Funny-looking, huh?  Python lets us make function calls where the arguments start with `**`, in which case a dictionary is expected afterward, to assign values to parameters by name.

Now let's create a bunch of generators that generate test cases against a given class.  A useful first ingredient is a function to create all possible dictionaries built from a set of keys and a set of available values.  Each key might get any of the values.

In [35]:
def all_dicts(keys, values):
    """Given a list of keys and an iterable of values, generate all possible dictionaries built with them."""
    
    if keys == []:
        yield {}
    else:
        for d in all_dicts(keys[1:], values):
            for v in values:
                d = d.copy()
                d[keys[0]] = v
                yield d

In [36]:
list(all_dicts(['A', 'B', 'C'], [1, 2]))

[{'C': 1, 'B': 1, 'A': 1},
 {'C': 1, 'B': 1, 'A': 2},
 {'C': 1, 'B': 2, 'A': 1},
 {'C': 1, 'B': 2, 'A': 2},
 {'C': 2, 'B': 1, 'A': 1},
 {'C': 2, 'B': 1, 'A': 2},
 {'C': 2, 'B': 2, 'A': 1},
 {'C': 2, 'B': 2, 'A': 2}]

Next, given a class, we can generate all possible calls to its methods, using only argument values drawn from a set that we provide.

In [37]:
def calls_for(cls, args):
    """Generates a finite stream of method calls that can be made against the given class.
    We only consider arguments drawn from the given iterable (args)."""
    
    for method in dir(cls):
        if not method.startswith('__'):
            params = dict(signature(getattr(cls, method)).parameters)
            del params['self'] # This parameter pops up by default,
                               # but Python already passes it for us automatically.
            for d in all_dicts(list(params), args):
                yield (method, d)

In [38]:
list(calls_for(BasicQueue, [0, 1]))

[('dequeue', {}), ('enqueue', {'v': 0}), ('enqueue', {'v': 1})]

Our second-last ingredient is a way to generate all call sequences of a given length.

In [39]:
def call_sequences_of_length(cls, length, args):
    """Generates a finite stream of method call sequences of the given length,
    made against the given class, using the provided arguments."""
    
    if length == 0:
        yield []
    else:
        for call in calls_for(cls, args):
            for rest in call_sequences_of_length(cls, length-1, args):
                yield [call] + rest

In [40]:
list(call_sequences_of_length(BasicQueue, 2, [0, 1]))

[[('dequeue', {}), ('dequeue', {})],
 [('dequeue', {}), ('enqueue', {'v': 0})],
 [('dequeue', {}), ('enqueue', {'v': 1})],
 [('enqueue', {'v': 0}), ('dequeue', {})],
 [('enqueue', {'v': 0}), ('enqueue', {'v': 0})],
 [('enqueue', {'v': 0}), ('enqueue', {'v': 1})],
 [('enqueue', {'v': 1}), ('dequeue', {})],
 [('enqueue', {'v': 1}), ('enqueue', {'v': 0})],
 [('enqueue', {'v': 1}), ('enqueue', {'v': 1})]]

Our examples so far generate finite sequences, but here is a generator for all possible test cases of arbitrary lengths (using only argument values from a provided set).  We generate the length-1 sequences first, then the length-2 sequences, and so on.  (Note that approach guarantees that, when we find a return-value disagreement between two calls, it always happens on the *last* call of the test case, since every prefix of the test case was already generated earlier in the sequence.)

In [41]:
def test_cases_for(cls, args):
    """Generates an infinite stream of test cases applicable to the given class,
    where arguments are drawn from the given iterable.
    A test case is a list of calls."""
    
    length = 1
    while True:
        yield from call_sequences_of_length(cls, length, args)
        length += 1

In [42]:
list(firstn(test_cases_for(BasicQueue, [0, 1]), 100))

[[('dequeue', {})],
 [('enqueue', {'v': 0})],
 [('enqueue', {'v': 1})],
 [('dequeue', {}), ('dequeue', {})],
 [('dequeue', {}), ('enqueue', {'v': 0})],
 [('dequeue', {}), ('enqueue', {'v': 1})],
 [('enqueue', {'v': 0}), ('dequeue', {})],
 [('enqueue', {'v': 0}), ('enqueue', {'v': 0})],
 [('enqueue', {'v': 0}), ('enqueue', {'v': 1})],
 [('enqueue', {'v': 1}), ('dequeue', {})],
 [('enqueue', {'v': 1}), ('enqueue', {'v': 0})],
 [('enqueue', {'v': 1}), ('enqueue', {'v': 1})],
 [('dequeue', {}), ('dequeue', {}), ('dequeue', {})],
 [('dequeue', {}), ('dequeue', {}), ('enqueue', {'v': 0})],
 [('dequeue', {}), ('dequeue', {}), ('enqueue', {'v': 1})],
 [('dequeue', {}), ('enqueue', {'v': 0}), ('dequeue', {})],
 [('dequeue', {}), ('enqueue', {'v': 0}), ('enqueue', {'v': 0})],
 [('dequeue', {}), ('enqueue', {'v': 0}), ('enqueue', {'v': 1})],
 [('dequeue', {}), ('enqueue', {'v': 1}), ('dequeue', {})],
 [('dequeue', {}), ('enqueue', {'v': 1}), ('enqueue', {'v': 0})],
 [('dequeue', {}), ('enqueue', 

The last ingredient is a way to run a test case against two classes, checking that they always agree on return values.

In [309]:
def run_test_case(cls1, cls2, case):
    """We are given two classes and a test case.
    Confirm that the two classes give the same answers on this test case."""
    
    v1 = cls1()
    v2 = cls2()
    for method, args in case:
        # The fanciness below enforces that both methods either return the same value
        # or both raise exceptions.  (For simplicity, we don't enforce that *the same*
        # exception is raised.)
        try:
            r1 = getattr(v1, method)(**args)
        except Exception:
            r1 = Exception
        
        try:
            r2 = getattr(v2, method)(**args)
        except Exception:
            r2 = Exception
        
        if r1 != r2:
            print('Test case failed:', case)
            assert False

In [310]:
run_test_case(BasicQueue, FancyQueue, [('enqueue', {'v': 0}), ('dequeue', {})])

Now it is easy to run our two classes head-to-head for as many distinct test cases as we like.

In [311]:
for case in firstn(test_cases_for(BasicQueue, [0, 1]), 100): run_test_case(BasicQueue, FancyQueue, case)

So far so good!  We could even go further and compare the two classes on *all* possible test cases (for our chosen set of arguments).  Of course this comparison will never finish if the classes truly match, but you could leave it running overnight to keep building confidence!  Notice how here it is important that a generator needn't write all its outputs into a list somewhere.  Even when there are only finitely many outputs, it could take up a lot of memory storing them all somewhere.  Our approach with generators materializes each test case as it is needed, taking up very little space.

In [312]:
#for case in test_cases_for(BasicQueue, range(10)): run_test_case(BasicQueue, FancyQueue, case)
# Warning: uncommenting the prior line leads to intentional infinite execution!